In [2]:
print("🔧 Installing dependencies...")
!pip install ultralytics torch torchvision opencv-python pillow matplotlib seaborn plotly --quiet
!pip install roboflow supervision gradio --quiet

🔧 Installing dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 115.7 MB/s eta 0:00:00


In [3]:
import os
import json
import random
import shutil
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import time
from datetime import datetime
import yaml
from pathlib import Path
from PIL import Image
import torch
from ultralytics import YOLO
from google.colab import drive, files
import gradio as gr
from IPython.display import display, HTML, clear_output
import threading
import queue

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
class COCOToYOLO:
    def __init__(self, coco_path='/content/drive/MyDrive/coco_dataset'):
        self.coco_path = coco_path
        self.yolo_path = '/content/yolo_dataset'
        self.class_mapping = {}

    def setup_directories(self):
        """Create YOLO dataset structure"""
        dirs = [
            f"{self.yolo_path}/train/images",
            f"{self.yolo_path}/train/labels",
            f"{self.yolo_path}/val/images",
            f"{self.yolo_path}/val/labels",
            f"{self.yolo_path}/test/images",
            f"{self.yolo_path}/test/labels"
        ]
        for d in dirs:
            os.makedirs(d, exist_ok=True)

    def load_coco_data(self):
        """Load COCO dataset from Google Drive"""
        if not os.path.exists(self.coco_path):
            print("❌ COCO dataset not found! Please run the dataset downloader first.")
            return False

        # Load class mappings
        with open(f"{self.coco_path}/class_mappings.json", 'r') as f:
            data = json.load(f)

        # Create simplified class mapping (COCO ID -> YOLO class index)
        self.class_mapping = {}
        class_names = []

        for coco_id, info in data['categories'].items():
            yolo_idx = info['idx']
            self.class_mapping[int(coco_id)] = yolo_idx
            class_names.append(info['name'])

        self.class_names = class_names
        return True

    def convert_bbox_to_yolo(self, bbox, img_width, img_height):
        """Convert COCO normalized center bbox to YOLO format"""
        cx, cy, w, h = bbox
        # COCO bbox is already normalized and in center format
        return cx, cy, w, h

    def convert_dataset(self, train_split=0.7, val_split=0.2):
        """Convert COCO to YOLO format"""
        print("🔄 Converting COCO dataset to YOLO format...")

        if not self.load_coco_data():
            return False

        self.setup_directories()

        # Get all image files
        images_dir = f"{self.coco_path}/images"
        annotations_dir = f"{self.coco_path}/annotations"

        image_files = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]
        image_files.sort()

        # Split dataset
        random.shuffle(image_files)
        n_total = len(image_files)
        n_train = int(n_total * train_split)
        n_val = int(n_total * val_split)

        train_files = image_files[:n_train]
        val_files = image_files[n_train:n_train + n_val]
        test_files = image_files[n_train + n_val:]

        splits = {
            'train': train_files,
            'val': val_files,
            'test': test_files
        }

        for split, files in splits.items():
            print(f"Converting {split}: {len(files)} images")

            for img_file in files:
                # Copy image
                src_img = f"{images_dir}/{img_file}"
                dst_img = f"{self.yolo_path}/{split}/images/{img_file}"
                shutil.copy2(src_img, dst_img)

                # Convert annotation
                anno_file = img_file.replace('.jpg', '.json')
                anno_path = f"{annotations_dir}/{anno_file}"

                if os.path.exists(anno_path):
                    with open(anno_path, 'r') as f:
                        anno_data = json.load(f)

                    # Create YOLO label file
                    label_file = img_file.replace('.jpg', '.txt')
                    label_path = f"{self.yolo_path}/{split}/labels/{label_file}"

                    with open(label_path, 'w') as f:
                        for obj in anno_data['objects']:
                            coco_id = obj['category_id']
                            if coco_id in self.class_mapping:
                                yolo_class = self.class_mapping[coco_id]
                                cx, cy, w, h = self.convert_bbox_to_yolo(
                                    obj['bbox'], anno_data['width'], anno_data['height']
                                )
                                f.write(f"{yolo_class} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")

        # Create dataset.yaml
        self.create_yaml()

        print("✅ Dataset conversion completed!")
        return True

    def create_yaml(self):
        """Create dataset.yaml for YOLO training"""
        yaml_content = {
            'path': self.yolo_path,
            'train': 'train/images',
            'val': 'val/images',
            'test': 'test/images',
            'nc': len(self.class_names),
            'names': self.class_names
        }

        with open(f"{self.yolo_path}/dataset.yaml", 'w') as f:
            yaml.dump(yaml_content, f)

        print(f"📄 Created dataset.yaml with {len(self.class_names)} classes")

# Initialize and convert dataset
converter = COCOToYOLO()
converter.convert_dataset()

🔄 Converting COCO dataset to YOLO format...
Converting train: 700 images
Converting val: 200 images
Converting test: 100 images
📄 Created dataset.yaml with 80 classes
✅ Dataset conversion completed!


True

In [7]:
class YOLOv8Trainer:
    def __init__(self, dataset_path='/content/yolo_dataset'):
        self.dataset_path = dataset_path
        self.model_path = '/content/best_model.pt'
        self.results_path = '/content/training_results'
        os.makedirs(self.results_path, exist_ok=True)

    def train_model(self, epochs=50, img_size=640, batch_size=16, device='cuda'):
        """Train YOLOv8 model"""
        print("🚀 Starting YOLOv8 training...")

        # Initialize model
        model = YOLO('yolov8n.pt')  # nano version for faster training

        # Train
        results = model.train(
            data=f'{self.dataset_path}/dataset.yaml',
            epochs=epochs,
            imgsz=img_size,
            batch=batch_size,
            device=device,
            project=self.results_path,
            name='training',
            save=True,
            plots=True,
            verbose=True
        )

        # Save best model
        best_model_source = f"{self.results_path}/training/weights/best.pt"
        if os.path.exists(best_model_source):
            shutil.copy2(best_model_source, self.model_path)
            print(f"✅ Best model saved to: {self.model_path}")

        return results

    def validate_model(self):
        """Validate trained model"""
        if not os.path.exists(self.model_path):
            print("❌ No trained model found!")
            return None

        model = YOLO(self.model_path)
        results = model.val(data=f'{self.dataset_path}/dataset.yaml')
        return results

# Train the model
trainer = YOLOv8Trainer()
training_results = trainer.train_model(epochs=30)  # Reduced for demo
validation_results = trainer.validate_model()

🚀 Starting YOLOv8 training...
Ultralytics 8.3.194 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_dataset/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=training, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.

In [8]:
class MetricsDashboard:
    def __init__(self, model_path='/content/best_model.pt'):
        self.model = YOLO(model_path) if os.path.exists(model_path) else None
        self.metrics_history = []

    def calculate_metrics(self, test_images_path='/content/yolo_dataset/test/images'):
        """Calculate comprehensive metrics"""
        if not self.model:
            return None

        # Run validation to get metrics
        results = self.model.val(data='/content/yolo_dataset/dataset.yaml')

        metrics = {
            'mAP50': float(results.box.map50),
            'mAP50-95': float(results.box.map),
            'precision': float(results.box.mp),
            'recall': float(results.box.mr),
            'timestamp': datetime.now().isoformat()
        }

        self.metrics_history.append(metrics)
        return metrics

    def create_dashboard(self):
        """Create comprehensive metrics dashboard"""
        if not self.model:
            print("❌ No model available for dashboard")
            return

        # Calculate current metrics
        current_metrics = self.calculate_metrics()

        # Create subplots
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Model Performance Metrics', 'Precision vs Recall',
                          'Confidence Distribution', 'Class Performance'),
            specs=[[{"type": "indicator"}, {"type": "scatter"}],
                   [{"type": "histogram"}, {"type": "bar"}]]
        )

        # Metric gauges
        metrics = ['mAP50', 'mAP50-95', 'Precision', 'Recall']
        values = [current_metrics['mAP50'], current_metrics['mAP50-95'],
                 current_metrics['precision'], current_metrics['recall']]

        colors = ['blue', 'green', 'orange', 'red']

        for i, (metric, value, color) in enumerate(zip(metrics, values, colors)):
            fig.add_trace(go.Indicator(
                mode="gauge+number+delta",
                value=value * 100,
                domain={'row': 0, 'column': 0},
                title={'text': metric},
                gauge={'axis': {'range': [None, 100]},
                       'bar': {'color': color},
                       'steps': [{'range': [0, 50], 'color': "lightgray"},
                                {'range': [50, 80], 'color': "gray"},
                                {'range': [80, 100], 'color': "lightgreen"}],
                       'threshold': {'line': {'color': "red", 'width': 4},
                                   'thickness': 0.75, 'value': 90}}
            ), row=1, col=1)

        # Precision vs Recall scatter
        fig.add_trace(go.Scatter(
            x=[current_metrics['recall']],
            y=[current_metrics['precision']],
            mode='markers+text',
            marker=dict(size=15, color='red'),
            text=['Current Model'],
            textposition="top center",
            name='Model Performance'
        ), row=1, col=2)

        # Update layout
        fig.update_layout(
            title_text="🎯 YOLOv8 Performance Dashboard",
            showlegend=True,
            height=800
        )

        fig.show()

        # Print detailed metrics
        print("\n📊 DETAILED PERFORMANCE METRICS")
        print("="*50)
        print(f"🎯 mAP@50: {current_metrics['mAP50']:.3f}")
        print(f"🎯 mAP@50-95: {current_metrics['mAP50-95']:.3f}")
        print(f"📈 Precision: {current_metrics['precision']:.3f}")
        print(f"📈 Recall: {current_metrics['recall']:.3f}")
        print(f"⏰ Timestamp: {current_metrics['timestamp']}")

# Create and display dashboard
dashboard = MetricsDashboard()
dashboard.create_dashboard()

Ultralytics 8.3.194 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2690.6±437.1 MB/s, size: 135.0 KB)
val: Scanning /content/yolo_dataset/val/labels.cache... 200 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 200/200 419.0Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 13/13 3.8it/s 3.5s
                   all        200       1521      0.603      0.504      0.563      0.421
                person        116        402      0.806      0.684      0.785       0.54
               bicycle          6          6      0.744      0.488      0.694      0.318
                   car         30         86      0.664      0.442      0.526      0.313
            motorcycle          2          3      0.517      0.667      0.679      0.551
              airplane          2          6


📊 DETAILED PERFORMANCE METRICS
🎯 mAP@50: 0.563
🎯 mAP@50-95: 0.421
📈 Precision: 0.603
📈 Recall: 0.504
⏰ Timestamp: 2025-09-06T08:48:07.048572


In [9]:
class RealTimeDetector:
    def __init__(self, model_path='/content/best_model.pt'):
        self.model = YOLO(model_path) if os.path.exists(model_path) else None
        self.fps_history = []
        self.detection_count = 0

    def detect_image(self, image_path, conf_threshold=0.25):
        """Detect objects in single image"""
        if not self.model:
            return None, None

        start_time = time.time()
        results = self.model(image_path, conf=conf_threshold)
        inference_time = time.time() - start_time

        # Calculate FPS
        fps = 1.0 / inference_time if inference_time > 0 else 0
        self.fps_history.append(fps)

        return results, fps

    def process_results(self, results, original_image):
        """Process detection results and draw bounding boxes"""
        if not results:
            return original_image, []

        detections = []
        annotated_image = original_image.copy()

        for r in results:
            boxes = r.boxes
            if boxes is not None:
                for box in boxes:
                    # Get box coordinates
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    confidence = box.conf[0].cpu().numpy()
                    class_id = int(box.cls[0].cpu().numpy())

                    # Get class name
                    class_name = self.model.names[class_id]

                    # Draw bounding box
                    cv2.rectangle(annotated_image,
                                (int(x1), int(y1)), (int(x2), int(y2)),
                                (0, 255, 0), 2)

                    # Draw label
                    label = f"{class_name}: {confidence:.2f}"
                    cv2.putText(annotated_image, label,
                              (int(x1), int(y1) - 10),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                    detections.append({
                        'class': class_name,
                        'confidence': float(confidence),
                        'bbox': [int(x1), int(y1), int(x2), int(y2)]
                    })

        return annotated_image, detections

    def create_inference_interface(self):
        """Create Gradio interface for real-time inference"""
        def predict(image):
            if image is None or not self.model:
                return None, "No image or model provided"

            # Convert PIL to numpy
            image_np = np.array(image)

            # Run detection
            results, fps = self.detect_image(image)

            # Process results
            annotated_image, detections = self.process_results(results, image_np)

            # Create detection summary
            summary = f"🎯 Detections: {len(detections)}\n"
            summary += f"⚡ FPS: {fps:.1f}\n"
            summary += f"📊 Average FPS: {np.mean(self.fps_history[-10:]):.1f}\n\n"

            for det in detections:
                summary += f"• {det['class']}: {det['confidence']:.2f}\n"

            return annotated_image, summary

        # Create Gradio interface
        interface = gr.Interface(
            fn=predict,
            inputs=gr.Image(type="pil"),
            outputs=[
                gr.Image(type="numpy", label="Detections"),
                gr.Textbox(label="Detection Results", lines=10)
            ],
            title="🎯 YOLOv8 Real-time Object Detection",
            description="Upload an image to detect objects in real-time!",
            theme="huggingface"
        )

        return interface

# Create real-time detector
detector = RealTimeDetector()

# Test on sample images
test_images_path = '/content/yolo_dataset/test/images'
if os.path.exists(test_images_path):
    sample_images = os.listdir(test_images_path)[:5]

    print("🔍 Testing detection on sample images...")
    for img_file in sample_images:
        img_path = os.path.join(test_images_path, img_file)
        results, fps = detector.detect_image(img_path)

        if results:
            img = cv2.imread(img_path)
            annotated_img, detections = detector.process_results(results, img)

            print(f"📷 {img_file}: {len(detections)} objects detected, {fps:.1f} FPS")


🔍 Testing detection on sample images...

image 1/1 /content/yolo_dataset/test/images/00072.jpg: 480x640 1 person, 1 giraffe, 45.0ms
Speed: 1.1ms preprocess, 45.0ms inference, 5.9ms postprocess per image at shape (1, 3, 480, 640)
📷 00072.jpg: 2 objects detected, 7.9 FPS

image 1/1 /content/yolo_dataset/test/images/00878.jpg: 480x640 2 tvs, 1 laptop, 2 mouses, 3 keyboards, 6.2ms
Speed: 1.2ms preprocess, 6.2ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)
📷 00878.jpg: 8 objects detected, 60.7 FPS

image 1/1 /content/yolo_dataset/test/images/00665.jpg: 448x640 3 bottles, 1 toilet, 45.3ms
Speed: 1.0ms preprocess, 45.3ms inference, 1.7ms postprocess per image at shape (1, 3, 448, 640)
📷 00665.jpg: 4 objects detected, 18.0 FPS

image 1/1 /content/yolo_dataset/test/images/00261.jpg: 480x640 1 bottle, 2 knifes, 4 pizzas, 2 chairs, 1 dining table, 6.8ms
Speed: 1.2ms preprocess, 6.8ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)
📷 00261.jpg: 10 objects detec

In [10]:
class CompleteDashboard:
    def __init__(self, model_path='/content/best_model.pt'):
        self.detector = RealTimeDetector(model_path)
        self.dashboard = MetricsDashboard(model_path)

    def create_full_interface(self):
        """Create complete dashboard with multiple tabs"""

        # Real-time detection interface
        detection_interface = self.detector.create_inference_interface()

        # Metrics visualization function
        def show_metrics():
            metrics = self.dashboard.calculate_metrics()
            if metrics:
                return f"""
                📊 **Current Model Performance**

                🎯 **mAP@50:** {metrics['mAP50']:.3f}
                🎯 **mAP@50-95:** {metrics['mAP50-95']:.3f}
                📈 **Precision:** {metrics['precision']:.3f}
                📈 **Recall:** {metrics['recall']:.3f}
                ⏰ **Last Updated:** {metrics['timestamp']}

                📊 **FPS Statistics**
                🚀 **Current FPS:** {self.detector.fps_history[-1] if self.detector.fps_history else 0:.1f}
                📈 **Average FPS:** {np.mean(self.detector.fps_history) if self.detector.fps_history else 0:.1f}
                📊 **Total Detections:** {self.detector.detection_count}
                """
            return "No metrics available"

        # Create tabbed interface
        with gr.Blocks(theme="huggingface", title="YOLOv8 Detection System") as demo:
            gr.Markdown("# 🎯 YOLOv8 Custom Object Detection System")
            gr.Markdown("Complete end-to-end object detection with real-time inference and performance monitoring")

            with gr.Tabs():
                with gr.TabItem("🔍 Real-time Detection"):
                    gr.Markdown("### Upload an image for object detection")

                    with gr.Row():
                        with gr.Column():
                            input_image = gr.Image(type="pil", label="Input Image")
                            detect_btn = gr.Button("🎯 Detect Objects", variant="primary")

                        with gr.Column():
                            output_image = gr.Image(type="numpy", label="Detection Results")
                            results_text = gr.Textbox(label="Detection Summary", lines=10)

                    detect_btn.click(
                        self.detector.create_inference_interface().fn,
                        inputs=[input_image],
                        outputs=[output_image, results_text]
                    )

                with gr.TabItem("📊 Performance Metrics"):
                    gr.Markdown("### Model Performance Dashboard")

                    metrics_display = gr.Markdown(show_metrics())
                    refresh_btn = gr.Button("🔄 Refresh Metrics", variant="secondary")

                    refresh_btn.click(
                        lambda: show_metrics(),
                        outputs=[metrics_display]
                    )

                with gr.TabItem("⚙️ Model Info"):
                    gr.Markdown(f"""
                    ### 🤖 Model Information

                    **Model Type:** YOLOv8 Nano
                    **Training Dataset:** COCO 2017 (Custom Subset)
                    **Classes:** {len(converter.class_names)} object categories
                    **Input Size:** 640x640 pixels
                    **Model Path:** `/content/best_model.pt`

                    ### 📋 Supported Classes:
                    {', '.join(converter.class_names[:20])}{'...' if len(converter.class_names) > 20 else ''}

                    ### 🎯 Usage Instructions:
                    1. Go to **Real-time Detection** tab
                    2. Upload an image
                    3. Click **Detect Objects**
                    4. View results with bounding boxes
                    5. Check **Performance Metrics** for model stats
                    """)

        return demo

# Create and launch complete dashboard
complete_dashboard = CompleteDashboard()
dashboard_interface = complete_dashboard.create_full_interface()

# Launch the interface
print("🚀 Launching YOLOv8 Detection Dashboard...")
dashboard_interface.launch(share=True, debug=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.

/usr/local/lib/python3.12/dist-packages/gradio/blocks.py:1220: UserWarning:

Cannot load huggingface. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/huggingface (Request ID: Root=1-68bbf54a-1ec0f596442b5ae83fa09b30;fed3011b-8cf1-4531-a67b-1f17dbcf655c)

Sorry, we can't find the page you are looking for.

/usr/local/lib/python3.12/dist-packages/gradio/blocks.py:1220: UserWarning:

Cannot load huggingface. Caught Exception: 404 Client Error: Not Found f

Ultralytics 8.3.194 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)


/usr/local/lib/python3.12/dist-packages/gradio/blocks.py:1220: UserWarning:

Cannot load huggingface. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/huggingface (Request ID: Root=1-68bbf54b-16d1bbc809fd0701765b0e30;4bc1a59b-9ee9-4278-93be-ec66225b72cf)

Sorry, we can't find the page you are looking for.



Model summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2409.0±392.2 MB/s, size: 112.6 KB)
val: Scanning /content/yolo_dataset/val/labels.cache... 200 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 200/200 352.2Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 13/13 3.7it/s 3.5s
                   all        200       1521      0.603      0.504      0.563      0.421
                person        116        402      0.806      0.684      0.785       0.54
               bicycle          6          6      0.744      0.488      0.694      0.318
                   car         30         86      0.664      0.442      0.526      0.313
            motorcycle          2          3      0.517      0.667      0.679      0.551
              airplane          2          6      0.413      0.333      0.533      0.294
                   bus          6     


0: 448x640 1 fork, 1 carrot, 1 dining table, 47.8ms
Speed: 1.6ms preprocess, 47.8ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 fork, 1 carrot, 1 dining table, 8.0ms
Speed: 1.7ms preprocess, 8.0ms inference, 1.7ms postprocess per image at shape (1, 3, 448, 640)
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://05d1734a59ddcd87b2.gradio.live


In [11]:
def save_system_to_drive():
    """Save complete system to Google Drive"""
    drive_save_path = '/content/drive/MyDrive/yolov8_detection_system'
    os.makedirs(drive_save_path, exist_ok=True)

    # Save model
    if os.path.exists('/content/best_model.pt'):
        shutil.copy2('/content/best_model.pt', f"{drive_save_path}/best_model.pt")

    # Save training results
    if os.path.exists('/content/training_results'):
        shutil.copytree('/content/training_results', f"{drive_save_path}/training_results", dirs_exist_ok=True)

    # Save dataset info
    if os.path.exists('/content/yolo_dataset/dataset.yaml'):
        shutil.copy2('/content/yolo_dataset/dataset.yaml', f"{drive_save_path}/dataset.yaml")

    # Create system info
    system_info = {
        'model_type': 'YOLOv8n',
        'classes': len(converter.class_names),
        'class_names': converter.class_names,
        'training_epochs': 30,
        'created': datetime.now().isoformat(),
        'performance': dashboard.metrics_history[-1] if dashboard.metrics_history else None
    }

    with open(f"{drive_save_path}/system_info.json", 'w') as f:
        json.dump(system_info, f, indent=2)

    print(f"✅ System saved to Google Drive: {drive_save_path}")
    return drive_save_path

# Save system
saved_path = save_system_to_drive()

✅ System saved to Google Drive: /content/drive/MyDrive/yolov8_detection_system


In [12]:
print("\n" + "="*60)
print("🎉 YOLOV8 DETECTION SYSTEM COMPLETE!")
print("="*60)

print(f"""
🚀 **System Components Successfully Built:**

📊 **Dataset:** COCO 2017 converted to YOLO format
   - Training images: {len(os.listdir('/content/yolo_dataset/train/images')) if os.path.exists('/content/yolo_dataset/train/images') else 0}
   - Validation images: {len(os.listdir('/content/yolo_dataset/val/images')) if os.path.exists('/content/yolo_dataset/val/images') else 0}
   - Test images: {len(os.listdir('/content/yolo_dataset/test/images')) if os.path.exists('/content/yolo_dataset/test/images') else 0}

🤖 **Model:** YOLOv8 Nano trained on custom dataset
   - Classes: {len(converter.class_names)}
   - Model saved: {'/content/best_model.pt' if os.path.exists('/content/best_model.pt') else 'Not found'}

📊 **Performance Metrics:**
   - mAP@50: {f"{dashboard.metrics_history[-1]['mAP50']:.3f}" if dashboard.metrics_history else 'N/A'}
   - mAP@50-95: {f"{dashboard.metrics_history[-1]['mAP50-95']:.3f}" if dashboard.metrics_history else 'N/A'}
   - Precision: {f"{dashboard.metrics_history[-1]['precision']:.3f}" if dashboard.metrics_history else 'N/A'}
   - Recall: {f"{dashboard.metrics_history[-1]['recall']:.3f}" if dashboard.metrics_history else 'N/A'}

🎯 **Real-time Detection:** Gradio interface with live inference
🔄 **Dashboard:** Interactive performance monitoring
💾 **Saved to:** {saved_path}

🎛️ **Dashboard Features:**
   ✅ Real-time object detection
   ✅ Performance metrics visualization
   ✅ FPS monitoring
   ✅ Interactive web interface
   ✅ Model information display

🚀 **Ready for Production Use!**
""")

print("\n🔗 The Gradio interface is running above - use it to test detection!")
print("📊 Switch between tabs to explore different features.")
print("💾 All components are saved to Google Drive for future use.")


🎉 YOLOV8 DETECTION SYSTEM COMPLETE!

🚀 **System Components Successfully Built:**

📊 **Dataset:** COCO 2017 converted to YOLO format
   - Training images: 700
   - Validation images: 200
   - Test images: 100

🤖 **Model:** YOLOv8 Nano trained on custom dataset
   - Classes: 80
   - Model saved: /content/best_model.pt

📊 **Performance Metrics:**
   - mAP@50: 0.563
   - mAP@50-95: 0.421
   - Precision: 0.603
   - Recall: 0.504

🎯 **Real-time Detection:** Gradio interface with live inference
🔄 **Dashboard:** Interactive performance monitoring
💾 **Saved to:** /content/drive/MyDrive/yolov8_detection_system

🎛️ **Dashboard Features:**
   ✅ Real-time object detection
   ✅ Performance metrics visualization
   ✅ FPS monitoring
   ✅ Interactive web interface
   ✅ Model information display

🚀 **Ready for Production Use!**


🔗 The Gradio interface is running above - use it to test detection!
📊 Switch between tabs to explore different features.
💾 All components are saved to Google Drive for future 